# Team Classifier
From the observations, I have decided to create a model to classify a team's division rather than an individuals division.

In [40]:
import pandas as pd

In [41]:
df = pd.read_csv("Dataset/ultimate_college_championship.csv")
df.head()

,player,level,gender,division,team_name,Turns,Ds,Assists,Points,plus_minus,team_games,turns_per_game,ds_per_game,ast_per_game,pts_per_game,pls_mns_per_game
0,Jacques Nissen,Division 1,Men,Division 1 Men,Brown Brownian Motion,12,8,38,13,47,8,1.500000,1.000000,4.750000,1.625000,5.875
1,Cal Nightingale,Division 1,Men,Division 1 Men,Brown Brownian Motion,3,0,12,27,36,8,0.375000,0.000000,1.500000,3.375000,4.500
2,Faye Burdick,Division 1,Women,Division 1 Women,Colorado Quandary,6,12,16,13,35,7,0.857143,1.714286,2.285714,1.857143,5.000
3,Matthew Gregor,Division 3,Men,Division 3 Men,Franciscan Fatal,2,6,3,26,33,6,0.333333,1.000000,0.500000,4.333333,5.500
4,Frankie Saraniti,Division 3,Women,Division 3 Women,Carleton Eclipse,11,15,12,17,33,6,1.833333,2.500000,2.000000,2.833333,5.500


In [42]:
# Split by gender and get team ids
male_df = df[df['gender'] == 'Men'].drop(columns='team_games')
female_df = df[df['gender'] == 'Women'].drop(columns='team_games')
team_divs = df[['team_name', 'level', 'gender']].drop_duplicates().set_index(keys='team_name')

In [43]:
male_teams_mean = male_df.groupby('team_name').mean(numeric_only=True)
male_teams_mean.columns = [col + ' mean' for col in male_teams_mean.columns]
male_teams_stdev = male_df.groupby('team_name').std(numeric_only=True)
male_teams_stdev.columns = [col + ' stdev' for col in male_teams_stdev.columns]
male_teams_all = male_teams_mean.join(male_teams_stdev)
male_teams_all = male_teams_all.join(team_divs['level'])
male_teams_all.head()

,Turns mean,Ds mean,Assists mean,Points mean,plus_minus mean,turns_per_game mean,ds_per_game mean,ast_per_game mean,pts_per_game mean,pls_mns_per_game mean,...,Ds stdev,Assists stdev,Points stdev,plus_minus stdev,turns_per_game stdev,ds_per_game stdev,ast_per_game stdev,pts_per_game stdev,pls_mns_per_game stdev,level
team_name,,,,,,,,,,,,,,,,,,,,,
Alabama-Huntsville Nightmares,3.260870,1.521739,2.260870,2.304348,2.826087,0.652174,0.304348,0.452174,0.460870,0.565217,...,2.371604,3.816440,2.835406,4.302771,0.900944,0.474321,0.763288,0.567081,0.860554,Division 1
Bates Orange Whip,5.235294,1.647059,4.588235,4.588235,5.588235,0.872549,0.274510,0.764706,0.764706,0.931373,...,2.644361,10.099869,4.925175,8.463590,1.231149,0.440727,1.683311,0.820863,1.410598,Division 3
Berry Bucks,3.653846,1.461538,3.000000,3.000000,3.807692,0.608974,0.243590,0.500000,0.500000,0.634615,...,1.838059,6.835203,4.758151,6.079600,0.921282,0.306343,1.139200,0.793025,1.013267,Division 3
Brown Brownian Motion,3.480000,1.440000,4.520000,4.560000,7.040000,0.435000,0.180000,0.565000,0.570000,0.880000,...,1.980741,10.364684,7.112665,13.308769,0.744004,0.247593,1.295585,0.889083,1.663596,Division 1
Cal Poly-SLO Slocore,3.625000,1.791667,4.208333,4.208333,6.583333,0.517857,0.255952,0.601190,0.601190,0.940476,...,2.021174,7.672532,4.969290,8.662346,0.824226,0.288739,1.096076,0.709899,1.237478,Division 1


In [44]:
def make_bipolar(cell):
    if cell == 'Division 1':
        return 1
    return -1

In [45]:
male_teams_all['level'] = male_teams_all['level'].apply(make_bipolar)

In [46]:
male_train = male_teams_all[0:28]
male_x_train = male_train.drop(columns='level')
male_y_train = male_train['level']

male_test = male_teams_all[28:]
male_x_test = male_test.drop(columns='level')
male_y_test = male_test['level']

In [47]:
def get_results(prediction, actual):
    data = {
        'Prediction': prediction,
        'Actual': actual
    }
    results = pd.DataFrame(data)
    num_correct = len(results[results['Prediction'] == results['Actual']])
    total = len(results)
    accuracy = round(num_correct/total * 100, 2)
    print(f"Accuracy: {accuracy}%")
    return results

In [48]:
from sklearn import tree

In [49]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(male_x_train, male_y_train)

In [50]:
predictions = clf.predict(male_x_test)
results = get_results(predictions, male_y_test)

Accuracy: 70.0%
